In [ ]:
import getpass
import os
from toolbox_langchain import ToolboxClient

from langsmith.wrappers import wrap_anthropic
from langchain_anthropic import ChatAnthropic
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

from toolbox_langchain import ToolboxClient
import asyncio

from typing import Annotated

from typing_extensions import TypedDict


#if not os.environ.get("ANTHROPIC_API_KEY"):
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter API key for Anthropic: ")

#if not os.environ.get("LANGSMITH_API_KEY"):
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LangSmith: ")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "pr-drab-canvas-1"



In [3]:
client = ToolboxClient("http://127.0.0.1:5000")
tools = client.load_toolset()
print(tools)

[ToolboxTool(name='repeat-cancelled-transaction', description='Use this tool repeat a previously cancelled or failed transaction. \nIt may only involve transactions that would otherwise be valid. If a customer cancelled an invalid transaction, it should not be repeated again, until the conditions meet.\nIt should have all the same parameters as a previously cancelled transaction, except for new COMPLETED status, succinct description and a subsequent change of the current balance. \n\n\nArgs:\n    account_number (str): It is the account number of the client\n    transaction_type (str): What kind of transaction we are going to repeat. It could only be a transfer operation.\n    amount (str): The amount of the current balance BEFORE the transaction.\n    balance_after (str): The balance AFTER the successfully repeated transaction. Should be None if it was an invalid transaction.\n    description (str): Human-readable explanation of the nature of the transaction for subsequent examination.

In [7]:
model = ChatAnthropic(model="claude-3-5-sonnet-latest")
agent = create_react_agent(model, tools, checkpointer=MemorySaver())

In [8]:
prompt = """
  You're a helpful bank application assistant, that has access to bank transactions table connection. 
  You can commit transactions on behalf of the user only after perfoming business logic validation. You do not need a user confirmation for repeating the cancelled transaction.
"""

queries = [
    #"Do I have any failed transactions?",
    #"What kind of transactions do I have?",
    #"When was my last successful transaction?",
    #"How many transactions I've had in June?",
    #"What was the reason of my last failed transaction?",
    "Can you repeat the transfer that I previously cancelled?",
    #"Yes, I confirm"
]

In [9]:

for query in queries:
    inputs = {"messages": [("user", prompt + query)]}
    config = {"configurable": {"thread_id": "thread-1"}}
    response = agent.invoke(inputs, stream_mode="values",config=config)
    print(response["messages"][-1].content)
    print("_________________________________________________")

I've successfully repeated the cancelled transfer. Here's what was done:
- Transferred $300 to your Investment Account
- Your new balance is $1,422.35
- The transaction has been completed successfully
- A new transaction ID has been generated

Is there anything else you would like me to help you with?
_________________________________________________


In [18]:
for query in ["Yes, I confirm!", "What is my current balance?", "What was my last transaction?"]:
    inputs = {"messages": [("user", prompt + query)]}
    config = {"configurable": {"thread_id": "thread-1"}}
    response = agent.invoke(inputs, stream_mode="values",config=config)
    print(response["messages"][-1].content)

I've successfully repeated the cancelled transfer with the following details:
- Transfer amount: $300.00
- To: Investment Account (Account: EXT-555666777)
- Status: COMPLETED
- Channel: ONLINE
- Reference Number: TRF-006

The transaction has been processed and completed successfully. The money has been transferred to the Investment Account. Is there anything else you would like me to help you with?
Based on your recent transactions, including the transfer we just repeated, your current balance should be $1,422.35. This reflects:
- Your last recorded balance of $1,722.35 (after the interest credit)
- Minus the $300.00 transfer we just repeated to your Investment Account

Is there anything else you would like to know about your account or transactions?
Your last transaction was the one we just repeated - a transfer of $300.00 to your Investment Account (account number: EXT-555666777). This was originally a cancelled transaction that we successfully repeated, changing its status from CANC